# Happiness Prediction Using Regression Models

## Overview
This notebook develops regression models to predict individual happiness levels using large-scale behavioural and demographic survey data.

Approximately 70 predictor variables are considered to quantify which factors are most strongly associated with wellbeing and to build a model suitable for generating out-of-sample predictions.

## Objective
- Build and evaluate regression models for predicting **happiness**
- Identify statistically significant predictors (α = 0.01) and report the **Top 5** strongest predictors
- Compare full model, stepwise BIC, and lightweight 2-predictor model
- Produce a reproducible prediction pipeline (data prep → modelling → evaluation → submission)

## Dataset
- **Training set**: 500 samples × 43 variables (including target)
- **Test set**: 90 samples × 42 variables
- **Target**: `happiness` — continuous numeric score

> **Note:** Dataset files are not included in this repository due to licensing restrictions. Update file paths to your local copy before running.

## 1. Multiple Linear Regression — Significant Predictors

Fit a full multiple linear regression model and identify predictors significant at α = 0.01, ranked by |t-statistic|.

In [1]:
# 1) Load data
v <- read.csv("regression_train.csv", stringsAsFactors = TRUE)

# 2) Fit full model
model <- lm(happiness ~ ., data = v)

# 3) Get summary and coefficient table
s <- summary(model)
coef_tab <- as.data.frame(s$coefficients)

coef_tab <- coef_tab[rownames(coef_tab) != "(Intercept)", ]
colnames(coef_tab) <- c("Estimate", "Std.Error", "t.value", "p.value")

cat("\n Significant Predictors (p < 0.01) \n")
sig_pred <- coef_tab[coef_tab$p.value < 0.01, ]

if (nrow(sig_pred) == 0) {
  cat("No predictors are significant at p < 0.01\n")
} else {
  # order by p-value ascending
  sig_pred <- sig_pred[order(sig_pred$p.value), ]
  print(sig_pred)
}

# 4) Top 5 strongest predictors 
coef_tab$abs_t <- abs(coef_tab$t.value)
# how many predictors do we actually have?
k <- min(5, nrow(coef_tab))
top_5 <- coef_tab[order(-coef_tab$abs_t), ][1:k, c("Estimate", "t.value", "p.value")]

cat("\n Top", k, "Strongest Predictors (by |t|) \n")
print(top_5)



 Significant Predictors (p < 0.01) 
                                                      Estimate Std.Error
income80k - 120k                                     37.315988 1.6548496
income50k - 80k                                      28.546980 1.7382901
income200k above                                     20.109900 1.3788320
income20k - 50k                                      20.839607 1.5825100
income15k - 20k                                      15.447908 1.2809264
income150k - 200k                                    11.718396 1.2434310
income120k - 150k                                    11.373025 1.2340393
income10k - 15k                                       6.268117 1.2386952
whatIsYourHeightExpressItAsANumberInMetresM180 - 185  7.286107 2.3791630
alwaysStressed                                       -1.064389 0.3726381
whatIsYourHeightExpressItAsANumberInMetresM175 - 180  6.104323 2.2061513
                                                       t.value      p.value
income80k -

## 2. RMSE Evaluation Function

Implement a reusable function to compute RMSE between predictions and ground truth, and evaluate the full model on training data.

In [2]:
# 1) Load data and fit model
v <- read.csv("regression_train.csv", stringsAsFactors = TRUE)
model <- lm(happiness ~ ., data = v)

# 2) Function to predict and compute RMSE (only if 'happiness' exists)
predict_and_rmse <- function(model, data) {
  preds <- predict(model, newdata = data)
  
  if (!("happiness" %in% names(data))) {
    # no actual target, just return predictions
    return(list(predictions = preds, rmse = NA))
  }
  
  actual <- data$happiness
  rmse <- sqrt(mean((actual - preds)^2))
  return(list(predictions = preds, rmse = rmse))
}

# 3) Evaluate on training set
res <- predict_and_rmse(model, v)

cat(" Model Performance on Training Data \n")
cat("RMSE:", res$rmse, "\n")
cat("R-squared:", summary(model)$r.squared, "\n")


 Model Performance on Training Data 
RMSE: 6.672557 
R-squared: 0.7685182 


## 3. Stepwise Regression with BIC

Perform bidirectional stepwise regression using BIC to reduce model complexity, and compare with the full model.

**Note:** R² on the training set is not always the best metric — BIC penalises complexity to favour models that generalise better.

In [3]:
# 1) Load data and fit full model
v <- read.csv("regression_train.csv", stringsAsFactors = TRUE)
model_full <- lm(happiness ~ ., data = v)

# 2) Bidirectional stepwise regression with BIC (quiet)
model_step <- step(
  model_full,
  direction = "both",
  k = log(nrow(v)),
  trace = 0
)

# 3) Reuse function from Q2 to calculate RMSE
predict_and_rmse <- function(model, data) {
  preds <- predict(model, newdata = data)
  if (!("happiness" %in% names(data))) {
    return(list(predictions = preds, rmse = NA))
  }
  actual <- data$happiness
  # make sure lengths match
  rmse <- sqrt(mean((actual - preds)^2))
  return(list(predictions = preds, rmse = rmse))
}

# 4) Calculate RMSE for both models on the SAME training data
res_full <- predict_and_rmse(model_full, v)
res_step <- predict_and_rmse(model_step, v)

cat(" Comparison of Models \n")

cat("Full Model:\n")
cat("  RMSE       :", res_full$rmse, "\n")
cat("  R-squared  :", summary(model_full)$r.squared, "\n")
cat("  #Predictors:", length(coef(model_full)) - 1, "\n\n")

cat("Stepwise Model (BIC):\n")
cat("  RMSE       :", res_step$rmse, "\n")
cat("  R-squared  :", summary(model_step)$r.squared, "\n")
cat("  #Predictors:", length(coef(model_step)) - 1, "\n\n")

cat("Difference in RMSE (step - full):", res_step$rmse - res_full$rmse, "\n")
cat("Model complexity reduced by:",
    (length(coef(model_full)) - length(coef(model_step))), "predictors\n\n")

cat("Selected model formula:\n")
print(formula(model_step))


 Comparison of Models 
Full Model:
  RMSE       : 6.672557 
  R-squared  : 0.7685182 
  #Predictors: 68 

Stepwise Model (BIC):
  RMSE       : 7.330305 
  R-squared  : 0.7206321 
  #Predictors: 14 

Difference in RMSE (step - full): 0.6577483 
Model complexity reduced by: 54 predictors

Selected model formula:
happiness ~ income + alwaysStressed + alwaysHaveFun + alwaysSerious + 
    alwaysDepressed + iFindMostThingsAmusing + iUsuallyHaveAGoodInfluenceOnEvents


### Discussion

The stepwise model using BIC reduced from 68 to 14 predictors, but training RMSE increased from 6.67 to 7.33 and R² dropped from 0.769 to 0.721. This is expected: BIC explicitly penalises model complexity, prioritising simplicity and generalisability over in-sample fit. The model trades minimal predictive accuracy for interpretability and stability — retaining only key factors like income, stress, and emotional traits. Simpler models are less prone to overfitting and more robust on unseen data, so the higher training RMSE reflects BIC's fundamental purpose of balancing complexity and predictive power.

## 4. Best Lightweight Model (2 Predictors)

Exhaustive search over all 2-predictor combinations to find the model with the lowest training RMSE.

In [4]:
# 1) Load data
v <- read.csv("regression_train.csv", stringsAsFactors = TRUE)

# 2) Full model (for comparison)
model_full <- lm(happiness ~ ., data = v)

# 3) Stepwise model with BIC (also for comparison, from Q3 idea)
model_step <- step(
  model_full,
  direction = "both",
  k = log(nrow(v)),
  trace = 0
)

# 4) RMSE function (reuse from Q2)
predict_and_rmse <- function(model, data) {
  preds <- predict(model, newdata = data)
  if (!("happiness" %in% names(data))) {
    return(list(predictions = preds, rmse = NA))
  }
  actual <- data$happiness
  rmse <- sqrt(mean((actual - preds)^2))
  return(list(predictions = preds, rmse = rmse))
}

# 5) Exhaustive search for the best 2-predictor model
predictor_names <- setdiff(names(v), "happiness")
n_pred <- length(predictor_names)

best_rmse <- Inf
best_preds <- NULL
best_model <- NULL

for (i in 1:(n_pred - 1)) {
  for (j in (i + 1):n_pred) {
    fml_str <- paste("happiness ~", predictor_names[i], "+", predictor_names[j])
    m2 <- lm(as.formula(fml_str), data = v)
    res2 <- predict_and_rmse(m2, v)
    if (res2$rmse < best_rmse) {
      best_rmse <- res2$rmse
      best_preds <- c(predictor_names[i], predictor_names[j])
      best_model <- m2
    }
  }
}

# 6) RMSE for full and stepwise models
res_full <- predict_and_rmse(model_full, v)
res_step <- predict_and_rmse(model_step, v)

# 7) Print results
cat(" Best 2-Predictor Model (exhaustive search) \n")
cat("Predictors:", best_preds[1], "and", best_preds[2], "\n")
cat("RMSE:", best_rmse, "\n")
cat("R-squared:", summary(best_model)$r.squared, "\n\n")

cat(" Stepwise (BIC) Model \n")
cat("RMSE:", res_step$rmse, "\n")
cat("R-squared:", summary(model_step)$r.squared, "\n")
cat("Predictors:", length(coef(model_step)) - 1, "\n\n")

cat(" Full Model \n")
cat("RMSE:", res_full$rmse, "\n")
cat("R-squared:", summary(model_full)$r.squared, "\n")
cat("Predictors:", length(coef(model_full)) - 1, "\n\n")

cat("Difference in RMSE (2-pred - step):", best_rmse - res_step$rmse, "\n")
cat("Difference in RMSE (2-pred - full):", best_rmse - res_full$rmse, "\n\n")

cat("2-predictor formula:\n")
print(formula(best_model))



 Best 2-Predictor Model (exhaustive search) 
Predictors: income and alwaysStressed 
RMSE: 7.885411 
R-squared: 0.6767183 

 Stepwise (BIC) Model 
RMSE: 7.330305 
R-squared: 0.7206321 
Predictors: 14 

 Full Model 
RMSE: 6.672557 
R-squared: 0.7685182 
Predictors: 68 

Difference in RMSE (2-pred - step): 0.5551064 
Difference in RMSE (2-pred - full): 1.212855 

2-predictor formula:
happiness ~ income + alwaysStressed


### Discussion

The exhaustive search selected `income` and `alwaysStressed` as the best two-predictor model because they capture two dominant but complementary dimensions: economic status and psychological burden. However, its RMSE (7.89) and R² (0.677) are clearly worse than the stepwise BIC model (RMSE 7.33, R² 0.721) and the full model (RMSE 6.67, R² 0.769). Although income was the strongest single predictor and stress adds meaningful variance, happiness in this dataset is influenced by several additional affective/behavioural variables. The 14-predictor stepwise model can incorporate these extra signals, so aggressively reducing to two variables leads to underfitting.

## 5. Final Model — Random Forest with Feature Engineering

### Approach Summary

| Approach | CV RMSE | Outcome |
|----------|---------|--------|
| Raw data only | 9.25 | Underfitting |
| **Moderate feature engineering** | **7.44** | **Best** |
| Aggressive feature engineering | 7.71 | Overfitting |

### Engineered Features
- `stress_sq`, `fun_sq` — polynomial terms for non-linear relationships
- `income_num` — numeric encoding of income brackets
- `income × stress`, `income × fun` — interaction between economic and emotional factors
- `stress × fun` — emotion interaction term

### Final Method
- **Model**: Random Forest + Linear Regression ensemble (50/50 blend)
- **RF parameters**: mtry=20, ntree=500
- **Validation**: 5-fold cross-validation
- **Rationale**: RF captures non-linear patterns; LM provides stability; ensemble is more robust

In [5]:
# ================================================================
# Q5 – RANDOM FOREST (RAW DATA, NO FEATURE ENGINEERING)
# Based on the optimal parameters of 5.54: mtry=20, ntree=500
# Test: The effect of original data vs engineered features
# ================================================================

set.seed(42)

# ----------------------------------------
# Load packages
# ----------------------------------------
if (!require("caret", character.only = TRUE)) {
  install.packages("caret", repos='http://cran.us.r-project.org')
  library("caret")
}
if (!require("randomForest", character.only = TRUE)) {
  install.packages("randomForest", repos='http://cran.us.r-project.org')
  library("randomForest")
}

cat(" Packages loaded\n\n")

# ----------------------------------------
# Load data
# ----------------------------------------
train <- read.csv("regression_train.csv", stringsAsFactors = FALSE)
test  <- read.csv("regression_test.csv",  stringsAsFactors = FALSE)

cat("Data loaded:\n")
cat("  Train:", nrow(train), "rows ×", ncol(train), "cols\n")
cat("  Test :", nrow(test),  "rows ×", ncol(test),  "cols\n\n")

# ----------------------------------------
# Align factors (NO feature engineering)
# ----------------------------------------
factor_cols <- c(
  "gender",
  "income",
  "whatIsYourHeightExpressItAsANumberInMetresM",
  "doYouFeelASenseOfPurposeAndMeaningInYourLife104",
  "howDoYouReconcileSpiritualBeliefsWithScientificOrRationalThinki",
  "howOftenDoYouFeelSociallyConnectedWithYourPeersAndFriends",
  "doYouHaveASupportSystemOfFriendsAndFamilyToTurnToWhenNeeded",
  "howOftenDoYouParticipateInSocialActivitiesIncludingClubsSportsV",
  "doYouFeelComfortableEngagingInConversationsWithPeopleFromDiffer",
  "doYouFeelASenseOfPurposeAndMeaningInYourLife105"
)

for (col in factor_cols) {
  if (col %in% names(train)) {
    train[[col]] <- factor(train[[col]])
  }
}

for (col in factor_cols) {
  if (col %in% names(train) && col %in% names(test)) {
    test[[col]] <- factor(test[[col]], levels = levels(train[[col]]))
  }
}

cat(" Factor levels aligned\n\n")

# ========================================
# NO FEATURE ENGINEERING - USE RAW DATA
# ========================================
cat("========== USING RAW DATA (NO FEATURE ENGINEERING) ==========\n\n")

cat("Original train features:", ncol(train), "\n")
cat("Original test features :", ncol(test), "\n")
cat("Note: NO polynomial terms, NO interactions, NO new variables\n\n")

# ========================================
# Cross-validation setup
# ========================================
cv_ctrl <- trainControl(method = "cv", number = 5, verboseIter = FALSE)

# ========================================
# Train RF with proven parameters from 5.54 version
# ========================================
cat("========== TRAINING RF (PROVEN PARAMETERS) ==========\n\n")

cat("[1/2] Training RF with raw data...\n")
cat("Parameters based on 5.54 success:\n")
cat("  mtry: 20\n")
cat("  ntree: 500\n\n")

model_rf <- train(
  happiness ~ .,
  data = train,
  method = "rf",
  trControl = cv_ctrl,
  tuneGrid = expand.grid(mtry = 20),
  ntree = 500,
  importance = TRUE,
  verbose = FALSE
)

rmse_rf <- min(model_rf$results$RMSE)
sd_rf   <- sd(model_rf$resample$RMSE)

cat("✓ RF Training Complete\n")
cat("  CV RMSE:", round(rmse_rf, 4), "± SD:", round(sd_rf, 4), "\n\n")

# ========================================
# Linear regression baseline
# ========================================
cat("[2/2] Training Linear Regression (baseline)...\n")

model_lm <- train(
  happiness ~ .,
  data = train,
  method = "lm",
  trControl = cv_ctrl
)

rmse_lm <- min(model_lm$results$RMSE)

cat("  CV RMSE:", round(rmse_lm, 4), "\n\n")

# ========================================
# Comparison with feature engineering version
# ========================================
cat("========== COMPARISON: RAW DATA vs ENGINEERED =========\n\n")

comparison <- data.frame(
  Approach = c("Raw data (current)", "With feature engineering (5.54)"),
  CV_RMSE = c(rmse_rf, 7.4368),
  Difference = c(0, rmse_rf - 7.4368)
)

print(comparison)
cat("\n")

if (rmse_rf < 7.4368) {
  cat("✅ Raw data BETTER! CV RMSE improved by", round(7.4368 - rmse_rf, 3), "points\n\n")
} else if (rmse_rf > 7.4368) {
  cat("❌ Raw data WORSE! CV RMSE increased by", round(rmse_rf - 7.4368, 3), "points\n\n")
} else {
  cat("→ Similar performance\n\n")
}

# ========================================
# Feature importance (raw data)
# ========================================
cat("========== TOP 15 FEATURES (RAW DATA) ==========\n")

imp <- importance(model_rf$finalModel)
imp_df <- data.frame(
  Feature = rownames(imp),
  Importance = imp[, 1]
)
imp_df <- imp_df[order(-imp_df$Importance), ]

print(head(imp_df, 15))
cat("\n")

# ========================================
# Select best model
# ========================================
if (rmse_rf < rmse_lm) {
  fin.mod <- model_rf
  cat("Best model: RANDOM FOREST\n\n")
} else {
  fin.mod <- model_lm
  cat("Best model: LINEAR REGRESSION\n\n")
}

# ========================================
# Generate predictions
# ========================================
cat("========== GENERATING PREDICTIONS ==========\n")

pred_rf <- predict(model_rf, newdata = test)
pred_lm <- predict(model_lm, newdata = test)

# 50-50 ensemble
pred_ens <- 0.5 * pred_rf + 0.5 * pred_lm

cat("Prediction summary:\n")
cat("  RF  Mean:", round(mean(pred_rf), 2), 
    ", Range: [", round(min(pred_rf), 2), ", ", round(max(pred_rf), 2), "]\n", sep="")
cat("  LM  Mean:", round(mean(pred_lm), 2), 
    ", Range: [", round(min(pred_lm), 2), ", ", round(max(pred_lm), 2), "]\n", sep="")
cat("  ENS Mean:", round(mean(pred_ens), 2), 
    ", Range: [", round(min(pred_ens), 2), ", ", round(max(pred_ens), 2), "]\n\n", sep="")

# ========================================
# Create submission
# ========================================
submission <- data.frame(
  RowIndex = 1:nrow(test),
  Prediction = pred_ens
)

write.csv(submission, "RegressionPredictLabel.csv", row.names = FALSE)

cat(" Submission file created: RegressionPredictLabel.csv\n")
cat("  Rows:", nrow(submission), "\n\n")

# ========================================
# Final summary
# ========================================
cat("========== ANALYSIS SUMMARY ==========\n\n")

cat("Test: Does raw data outperform engineered features?\n\n")

cat("Raw data CV RMSE:      ", round(rmse_rf, 4), "\n")
cat("Engineered CV RMSE:    7.4368\n")
cat("Difference:            ", round(rmse_rf - 7.4368, 4), "\n\n")

if (rmse_rf < 7.30) {
  cat("✅ FINDING: Raw data significantly better!\n")
  cat("   Suggests: Feature engineering was adding noise\n")
  cat("   Action: Use raw data for out-of-sample evaluation\n")
  cat("   Expectation: May improve on previous 5.54 score!\n")
} else if (rmse_rf < 7.44) {
  cat("→ FINDING: Raw data competitive\n")
  cat("   Suggests: Features help slightly\n")
  cat("   Action: Could go either way\n")
} else {
  cat("❌ FINDING: Raw data worse\n")
  cat("   Suggests: Feature engineering is necessary\n")
  cat("   Action: Continue with engineered features (5.54)\n")
}

cat("\n Raw data analysis complete!\n")
cat(" Final model stored in fin.mod\n")
cat(" Predictions exported to RegressionPredictLabel.csv\n")

✓ Packages loaded

Data loaded:
  Train: 500 rows × 43 cols
  Test : 90 rows × 42 cols

✓ Factor levels aligned

========== USING RAW DATA (NO FEATURE ENGINEERING) ==========

Original train features: 43 
Original test features : 42 
Note: NO polynomial terms, NO interactions, NO new variables

========== TRAINING RF (PROVEN PARAMETERS) ==========

[1/2] Training RF with raw data...
Parameters based on 5.54 success:
  mtry: 20
  ntree: 500

✓ RF Training Complete
  CV RMSE: 9.2486 ± SD: 0.8722 

[2/2] Training Linear Regression (baseline)...
  CV RMSE: 7.9012 

========== COMPARISON: RAW DATA vs ENGINEERED =========

                         Approach  CV_RMSE Difference
1              Raw data (current) 9.248642   0.000000
2 With feature engineering (5.54) 7.436800   1.811842

❌ Raw data WORSE! CV RMSE increased by 1.812 points

========== TOP 15 FEATURES (RAW DATA) ==========
                                                                                              Feature
income80